# 1. <a id='toc1_'></a>[Features Selection](#toc0_)

This notebook contains the code to select the most important features for the model. 

Here filter mothod is used to select the features. The features are selected using the `feature_importance_` attribute of the model. The features are then ranked and the top 10 features are selected. The selected features are then used to train the model.

Filter-based feature selection methods for unsupervised data typically rely on statistical measures or heuristic approaches to rank features based on their intrinsic characteristics rather than on a specific learning algorithm. Here are a few filter-based methods along with how you might associate each selected feature with its importance:

+ Variance Threshold</br>
Compute the variance of each feature. Features with low variance are less informative and can be removed.
Associate the variance value directly as the feature importance.
Correlation Coefficient:

+ Calculate the correlation coefficient between each pair of features.</br>
  Features highly correlated with other features might contain redundant information. You can select one of each highly correlated pair or remove one randomly.
Associate the absolute value of the correlation coefficient as the feature importance.
Mutual Information:

+ Measure the mutual information between each feature and the cluster labels.</br>
Features with high mutual information are more informative for clustering.
Associate the mutual information value as the feature importance.
Distance-based Methods:

+ Compute the distance between instances in the feature space and analyze the distribution of distances.</br>
Features that contribute to larger distances between instances might be more important for clustering.
Associate the distance measure (e.g., mean distance or median distance) as the feature importance.

+ ``sklearn.feature_selection`` module is used for feature selection/dimensionality reduction.
+ Goal:
  + Improve estimators accuracy scores
  + Avoiding overfitting
  + Reduce the computational cost
  + Improve the comprehensibility of the model
+ There are three main strategies:
  + Univariate statistics: Select the best features based on univariate statistical tests
  + Model-based selection: Use a supervised model to judge the importance of each feature
  + Iterative selection: Build a model on initial features and then iteratively remove the least important feature
+ Feature selection methods can also be categorised into:
  + Filter methods: Select features based on their scores in various statistical tests
  + Wrapper methods: Select features based on the performance of a model trained with the selected features
  + Embedded methods: Select features based on the importance of their contribution to the model
+ Feature selection can be done in four ways:
  + **SelectKBest**: Select features according to the k highest scores
  + **SelectPercentile**: Select features according to a percentile of the highest scores
  + **SelectFpr**: Select features based on a false positive rate test
  + **SelectFdr**: Select features based on an estimated false discovery rate
  + **SelectFwe**: Select features based on family-wise error 

**Table of contents**<a id='toc0_'></a>    
1. [Features Selection](#toc1_)    
1.1. [Dependencies and paths](#toc1_1_)    
1.2. [Load the data](#toc1_2_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1.1. <a id='toc1_1_'></a>[Dependencies and paths](#toc0_)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
## DEPENDENCIES >>>
import os
import sys
from typing import List, Tuple, Dict, Any, Optional, Callable, Union
from pathlib import Path

import joblib
from functools import partial

# Add root directory to path for imports >
root_dir = Path.cwd().resolve().parent
if root_dir.exists():
    sys.path.append(str(root_dir))
else:
    raise FileNotFoundError('Root directory not found')

# import custom libraries >
from src.load import load_multiple_trajectoryCollection_parallel_pickle as lmtp
from src.load import load_datasets, load_df_to_dataset
from src.traj_dataloader import (TrajectoryDataset, 
                                 create_dataloader, 
                                 separate_files_by_season, 
                                 split_data, 
                                 get_files,
                                 AISDataset,
                                 )
from src.scaler import CustomMinMaxScaler, reduce_resolution

from datetime import datetime, timedelta

import dotsi
import itertools
import pickle

import numpy as np
import pandas as pd
from pandas import DataFrame

# torch libraries >
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.io import read_image

# sklearn libraries >
import sklearn as sk
from sklearn.model_selection import (train_test_split, 
                                     GridSearchCV, 
                                     RandomizedSearchCV)#, HalvingGridSearchCV, HalvingRandomSearchCV)
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score 
# from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline, make_pipeline

# Features selection >
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import ( mutual_info_classif,
                                       SelectKBest,
                                       chi2,
                                       VarianceThreshold,
                                       RFE,
                                       )
from mlxtend.feature_selection import ExhaustiveFeatureSelector
from skfeature.function.similarity_based import fisher_score
from sklearn.neural_network import MLPRegressor

# Hyperopt >
import optuna

# Mask warnings
import warnings
warnings.filterwarnings("ignore")  # Mask all warnings
# from sklearn.exceptions import ConvergenceWarning
# warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Plot >
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots  # https://github.com/garrettj403/SciencePlots?tab=readme-ov-file
plt.style.use(['science', 'grid', 'notebook'])  # , 'ieee'

# Multiprocessing >
from concurrent.futures import ProcessPoolExecutor
from functools import partial

# Toy datasets >
from sklearn.datasets import load_iris  # Sample dataset

# %matplotlib inline
%matplotlib widget

In [ ]:
## FLAGS & GLOBAL VALUES >>>

# Down sample the resolution
DOWN_SAMPLE = False  # used with SCALE and SAVE_SCALE to save the scaled data: (if True) with down sampled resolution, or with (not False) not.

# Explore
EXPLORE = True

# Debug
DEBUG = True

# Develop
DEVELOP = True

# HYPERPARAMETER OPTIMISATION
HYPEROPT = True

if HYPEROPT:
    OPTUNA = False # Optimise using Optuna
    GRIDSEARCH = False  # Optimise using GridSearchCV
    RANDOMSEARCH = True  # Optimise using RandomizedSearchCV

# SAVE SELECTED FEATURES in root / models / selected_features
SAVE_SELECT_FEATURES = True

# WORKING SERVER
AVAILABLE_SERVERS = ['ZS', 'PLOEN', 'KIEL', 'WYK']
CURRENT_SERVER = AVAILABLE_SERVERS[1]

# seed
split_seed = 42

# If DOWN_SAMPLE, define the target time resolution
targeted_resolution_min = 1  # minute

# TODO: The following featues are corrupted by containing NaNs. Fix this. For now, these columns are dropped
corrupted_features = ["stopped", "abs_ccs", "curv"]


# Use up to 70% of the available cpu cores
n_jobs = joblib.cpu_count()
print("Number of CPUs available:", n_jobs)
if CURRENT_SERVER == 'ZS':
    n_jobs = int(0.9 * n_jobs)
else:
    n_jobs = int(0.7 * n_jobs)
print("Number of CPUs to use:", n_jobs)

In [ ]:
## PATHS >>>
# data dir
data_dir = root_dir / 'data'
data_dir = data_dir.resolve()
if not data_dir.exists():
    raise FileNotFoundError('Data directory not found')

if CURRENT_SERVER == 'ZS':
    # assets dir  # TODO: Used temporarly during the features seletion process. Remove this!
    assets_dir = data_dir / 'assets'
    assets_dir = assets_dir.resolve()
    if not assets_dir.exists():
        raise FileNotFoundError(f'Assets directory in {CURRENT_SERVER} not found')
else:
    # aistraj dir
    assets_dir = data_dir / 'local' / 'aistraj'
    assets_dir = assets_dir.resolve()
    if not assets_dir.exists():
        raise FileNotFoundError('Assets directory not found')

    # train-validate-test (tvt) dir
    tvt_assets_dir = assets_dir / 'tvt_assets'
    tvt_assets_dir = tvt_assets_dir.resolve()
    if not tvt_assets_dir.exists():
        raise FileNotFoundError('Train-Validate-Test Assets directory not found')

    # tvt: extended pickle dir
    tvt_extended_dir = tvt_assets_dir / 'extended'
    tvt_extended_dir = tvt_extended_dir.resolve()
    if not tvt_extended_dir.exists():
        raise FileNotFoundError('TVT Extended Pickled Data directory not found')

    # tvt: scaled pickle dir
    tvt_scaled_dir = tvt_assets_dir / 'scaled'
    tvt_scaled_dir = tvt_scaled_dir.resolve()
    if not tvt_scaled_dir.exists():
        raise FileNotFoundError('TVT Scaled Pickled Data directory not found')

    # tvt: logs dir
    tvt_logs_dir = tvt_assets_dir / 'logs'
    tvt_logs_dir = tvt_logs_dir.resolve()
    if not tvt_logs_dir.exists():
        raise FileNotFoundError('TVT logs directory not found')
    
# models dir
models_dir = root_dir / 'models'
models_dir = models_dir.resolve()
if not models_dir.exists():
    raise FileNotFoundError('Models directory not found')

# Selected Features dir
selected_features_dir = models_dir / 'selected_features'
selected_features_dir = selected_features_dir.resolve()
if not selected_features_dir.exists():
    raise FileNotFoundError('selected features directory not found')

## 1.2. <a id='toc1_2_'></a>[Load the data](#toc0_)

+ Select the paths of the scaled datasets

In [ ]:
import_paths = {'train': None, 'validate': None, 'test': None}

if DOWN_SAMPLE:
    import_paths = {
                    'train': tvt_scaled_dir / 'scaled_cleaned_downsampled_extended_train_df.parquet',
                    'validate': tvt_scaled_dir / 'scaled_cleaned_downsampled_extended_validate_df.parquet',
                    'test': tvt_scaled_dir / 'scaled_cleaned_downsampled_extended_test_df.parquet'
                    }
else:  
    if CURRENT_SERVER != 'ZS':
        import_paths = {
                        'train': tvt_scaled_dir / 'scaled_cleaned_extended_train_df.parquet',
                        'validate': tvt_scaled_dir / 'scaled_cleaned_extended_validate_df.parquet',
                        'test': tvt_scaled_dir / 'scaled_cleaned_extended_test_df.parquet'
                        }
    else:
        import_paths = {
                        'train': assets_dir / 'scaled_cleaned_extended_train_df.parquet',
                        'validate': assets_dir / 'scaled_cleaned_extended_validate_df.parquet',
                        'test': assets_dir / 'scaled_cleaned_extended_test_df.parquet'
                        }
        
# Assets container >
train_df, validate_df, test_df = None, None, None
assets = {'train': train_df, 'validate': validate_df, 'test': test_df}

+ Load the train set

In [ ]:
# %%time
# if not DEVELOP:  # Data is huge! don't use for exploring and developping
#     train_df = load_df_to_dataset(data_path=import_paths['train'], use_dask=False).data  # Load the train dataset

+ Load the validate set

In [ ]:
%%time
validate_df = load_df_to_dataset(import_paths['validate'], use_dask=False).data  # Load the validate dataset

In [ ]:
if EXPLORE:
    columns = validate_df.columns
    print(f"Num. Cols: {len(columns)}: {columns}")
    print()
    print(f"Num. Samples: {validate_df.shape[0]}")

In [ ]:
if EXPLORE:
    display(validate_df.describe())

In [ ]:
if EXPLORE:
    validate_df.info()

+ Load the test set

In [ ]:
# %%time
# if not DEVELOP:  # Data is huge! don't use for exploring and developping
#     test_df = load_df_to_dataset(import_paths['test'], use_dask=False).data  # Load the test dataset

+ Concat the datasets

In [ ]:
# # Concatenate the datasets >
asset_df = validate_df  # asset_df = pd.concat([train_df, validate_df, test_df], axis=0)

# # Sort the dataset by epoch >
# asset_df = asset_df.sort_values(by='epoch', ascending=True)

# # Reset the index >
# asset_df = asset_df.reset_index(drop=True)

# # Display the dataset's head >
# if EXPLORE:
#     asset_df.head()

## Filter-based features selection

In [ ]:
cols_not_to_study = ['epoch', 'datetime', 'obj_id', 'traj_id', 'stopped', 'curv']

# Check that the column in cols_not_to_study are in the dataset, otherwise remove them from the list >
cols_not_to_study = [col for col in cols_not_to_study if col in asset_df.columns]

print(f"Cols not to study: {cols_not_to_study}")


### MLP Regressor AE

MLPRegressor autoencoders provide a powerful framework for feature selection by leveraging the network's ability to learn meaningful representations of input data in a lower-dimensional space.

Using an MLPRegressor (Multi-layer Perceptron Regressor) as an autoencoder for feature selection involves utilizing the network's ability to learn and compress input data into a lower-dimensional representation. This compression process serves as a form of feature selection because it forces the network to learn the most important features needed to reconstruct the input data.

Here's how the process typically works:

1. **Input Data**: You start with a dataset containing a set of features, possibly with many dimensions.

2. **Autoencoder Architecture**: You define an MLPRegressor architecture with at least one hidden layer. The size of the hidden layer(s) is typically smaller than the input layer, which forces the network to learn a compressed representation of the input features.

3. **Training**: The autoencoder is trained using the same input data for both the input and output layers. The objective is to minimize the reconstruction error, i.e., the difference between the input and the output. During training, the network learns to map the input data to a lower-dimensional space and then reconstruct it back to its original form.

4. **Feature Importance**: After training, the hidden layers of the autoencoder contain representations of the input data. The activations of neurons in these hidden layers can be interpreted as feature importance scores. Features that contribute more to the reconstruction of the input data will have higher activations in the hidden layers.

5. **Selection**: Based on the learned representations, you can select features by considering the activations of neurons in the hidden layers. Features corresponding to neurons with higher activations are considered more important, and those with lower activations are considered less important. You can either directly use these activations as feature scores or apply further processing (e.g., normalization) to obtain more interpretable scores.

Using an MLPRegressor as an autoencoder for feature selection offers several advantages:

- **Non-linearity**: MLPs can capture complex relationships between features, allowing for more effective feature selection compared to linear methods.
- **Flexibility**: The architecture of the autoencoder can be customized based on the characteristics of the dataset, allowing for adaptability to different types of data.
- **Unsupervised Learning**: Autoencoders do not require labeled data for training, making them suitable for unsupervised feature selection tasks where labeled data may be limited or unavailable.

#### Define the Optuna objective function for the optimisation of ``threshold`` hyperparameter

In [ ]:
%%time
# Create a copy of the dataset and drop the columns not to study >
df = asset_df.drop(columns=cols_not_to_study)

+ Find the best threshold value for the variance threshold method using Optuna. Using the silhouette score with k-means clustering.
  > **NOTE**:</br> Assuming that the number of clusters is $10$.
    

In [ ]:
def feature_selection_autoencoder(df, n_hidden, activation='relu', solver='adam', max_iter=1000, random_state=split_seed) -> Tuple[MLPRegressor, pd.DataFrame]:
    """
    Perform feature selection using an MLPRegressor autoencoder.

    Args:
        df (DataFrame): The input dataframe containing the features.
        n_hidden (int): The number of hidden units in the autoencoder.
        activation (str, optional): The activation function to use in the autoencoder. Defaults to 'relu'.
        solver (str, optional): The solver to use in the autoencoder. Defaults to 'adam'.
        max_iter (int, optional): The maximum number of iterations for training the autoencoder. Defaults to 1000.
        random_state (int, optional): The random state for reproducibility. Defaults to 42.

    Returns:
        tuple: A tuple containing two lists - selected_features and feature_scores.
            - selected_features (list): The selected features based on their importance scores.
            - feature_scores (list): The importance scores of the selected features.
    """
    # Instantiate a place holder for the variance threshold method (vtm) selected features >
    fs_df = pd.DataFrame(columns=['selected_features', 'feature_importance'])
    
    # Feature selection using MLPRegressor autoencoder
    mlp_autoencoder = MLPRegressor(hidden_layer_sizes=(len(df.columns) // 2,), activation=activation, solver=solver, max_iter=max_iter, random_state=random_state)
    mlp_autoencoder.fit(df, df)
    encoded_features = mlp_autoencoder.predict(df)
    
    # Calculate reconstruction errors
    reconstruction_errors = np.mean(np.square(df.values - encoded_features), axis=0).reshape(1, -1)  # Reshape to 2D array

    # Normalize reconstruction errors using Normalizer
    normalizer = Normalizer()
    normalized_errors = normalizer.fit_transform(reconstruction_errors)
    
    # Extract normalized scores
    feature_scores = 1 - normalized_errors.flatten()
    
    # Sort features based on importance scores
    sorted_features = sorted(zip(df.columns, feature_scores), key=lambda x: x[1], reverse=True)
    
    # Normalize the reconstruction errors to get scores that represent the importance of each feature. 
    selected_features = [feat for feat, _ in sorted_features]
    feature_scores = [score for _, score in sorted_features]
    
    # put the data in fs_df >
    fs_df['selected_features'] = selected_features
    fs_df['feature_importance'] = feature_scores
    
    return mlp_autoencoder, fs_df

In [ ]:
def optimize_mlp_hyperparameters(df):
    """
    Optimize hyperparameters for a MLPRegressor model using GridSearchCV.

    Parameters:
    - df: The input DataFrame for training the model.

    Returns:
    - best_params: The best hyperparameters found by GridSearchCV.
    """
    
    # Hyperparameter optimization using GridSearchCV
    param_grid = {
        'mlp__hidden_layer_sizes': [(10,), (50,), (100,), (200,)],
        'mlp__activation': ['relu', 'tanh'],
        'mlp__solver': ['adam', 'lbfgs'],
        'mlp__max_iter': [500, 1000, 1500]
    }
    
    # Declare the MLP AE
    mlp = MLPRegressor(random_state=split_seed)
    # Declare the kMeans >
    n_samples = df.shape[0]
    kmeans = KMeans(n_clusters=min(30, n_samples//2), random_state=42)
    # Create the pipeline >
    pipeline = Pipeline([('mlp', mlp), ('kmeans', kmeans)])
    
    # Define a function to compute silhouette score
    def silhouette_scorer(estimator, X):
            labels = estimator.predict(X)
            return silhouette_score(X, labels)
    
    grid_search = GridSearchCV(estimator=pipeline, 
                               param_grid=param_grid, 
                               cv=None, 
                               scoring=silhouette_scorer,  # 'neg_mean_squared_error',
                               n_jobs=n_jobs,
                               verbose=2)
    grid_search.fit(df, df)
    
    best_params = grid_search.best_params_
    
    return best_params


In [18]:
%%time
best_params = optimize_mlp_hyperparameters(df)

best_n_hidden = best_params['mlp__hidden_layer_sizes'][0]
best_activation = best_params['mlp__activation']
best_solver = best_params['mlp__solver']
best_max_iter = best_params['mlp__max_iter']

In [ ]:
_, fs_df = feature_selection_autoencoder(df, best_n_hidden, best_activation, best_solver, best_max_iter)

# Free up memory
del df

In [ ]:
display(fs_df)

In [ ]:
# Save the selected features to the models directory >
if SAVE_SELECT_FEATURES:
    fs_df.to_csv(selected_features_dir / 'selected_features_mlp.csv', index=False)
    print("Selected Features saved to:", selected_features_dir / 'selected_features_mlp.csv')